# 1. MODULOS

In [1]:
import os
import glob
import pandas as pd

# 2. RUTAS Y VARIABLES

In [2]:
# Se define la ruta de la carpeta donde se encuentran los archivos SAT de todos los años
R_Sat_base = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SAT\SUBSIDIADO"

# Se define la ruta de la carpeta de salida para los consolidados anuales
R_Salida_base = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SAT\SUBSIDIADO\consolidado año"

# 3 UNIFICACION ARCHIVOS

In [3]:
# ----------------------------------
# EJECUCIÓN DEL FLUJO DE TRABAJO
# ----------------------------------

# Crear la carpeta de salida si no existe
if not os.path.exists(R_Salida_base):
    os.makedirs(R_Salida_base)

# Obtener una lista de los subdirectorios que representan los años (ej. "2018", "2019", etc.)
# Se filtran solo los directorios que contienen un número para evitar carpetas como 'consolidado año'
años = [d for d in os.listdir(R_Sat_base) if os.path.isdir(os.path.join(R_Sat_base, d)) and d.isdigit()]

# Iterar sobre cada año encontrado
for año in años:
    print(f"Procesando el año: {año}")
    
    # Se define la ruta de la carpeta para el año actual en el bucle
    R_Sat_año = os.path.join(R_Sat_base, año)
    
    # Se define el nombre del archivo de salida para el año actual
    SAT_SALIDA_año = f"SAT_EPS025_{año}.TXT"
    
    # Se construye la ruta completa del archivo de salida
    R_Salida_año = os.path.join(R_Salida_base, SAT_SALIDA_año)

    # Buscar todos los archivos .txt en la ruta del año y sus subcarpetas
    archivos = glob.glob(os.path.join(R_Sat_año, '**', '*.txt'), recursive=True)

    # Si no se encuentran archivos para el año, se salta al siguiente
    if not archivos:
        print(f"No se encontraron archivos .txt para el año {año}. Saltando.")
        continue

    dataframes = []

    for archivo in archivos:
        # Extraer la fecha del nombre del archivo
        nombre = os.path.basename(archivo)
        
        # El formato de los nombres de archivo varía. Se ha modificado la lógica para
        # manejar un formato más flexible, como 'EPS025_2018-3-16.txt' o 'SAT-2025-05-31.txt'
        
        partes = nombre.split('-')

        # Se verifica si el nombre de archivo se ajusta a alguno de los formatos esperados
        if len(partes) == 4:
            # Formato 'SAT-2025-05-31.txt'
            anio_archivo = partes[1]
            mes_archivo = partes[2]
            dia_archivo = partes[3].split('.')[0]
        elif len(partes) >= 3:
            # Formato 'EPS025_2018-3-16.txt'
            # Extraemos el año de la primera parte (ej. '2018' de 'EPS025_2018')
            parte_anio = partes[0]
            try:
                anio_archivo = parte_anio.split('_')[1]
                mes_archivo = partes[1]
                dia_archivo = partes[2].split('.')[0]
            except IndexError:
                # Si no se puede extraer el año, se ignora el archivo
                print(f"Alerta: El archivo '{nombre}' no tiene el formato esperado (no se pudo extraer el año). Este archivo será ignorado.")
                continue
        else:
            # Si el formato no coincide con ninguno, se ignora
            print(f"Alerta: El archivo '{nombre}' no tiene el formato esperado. Este archivo será ignorado.")
            continue  

        fecha_archivo = f"{dia_archivo.zfill(2)}-{mes_archivo.zfill(2)}-{anio_archivo}"

        try:
            # Se usa header=None para no ignorar la primera fila
            # Leer el archivo como texto, sin importar el tipo de dato
            df = pd.read_csv(archivo, sep='|', dtype=str, encoding='ansi', on_bad_lines='skip', header=None)
            
            # Agregar la columna de fecha al inicio
            df.insert(0, 'fecha_archivo', fecha_archivo)
            dataframes.append(df)
        except Exception as e:
            print(f"Error al leer el archivo '{nombre}': {e}. Este archivo será ignorado.")
            continue

    # Si no se pudo leer ningún dataframe, pasar al siguiente año
    if not dataframes:
        print(f"No se pudieron leer archivos válidos para el año {año}. Saltando.")
        continue

    # Unir todos los dataframes del año actual
    df_total = pd.concat(dataframes, ignore_index=True)

    # Eliminar los registros que son encabezados normativos
    # Se crea una máscara booleana para identificar los registros que coinciden con el patrón
    # Se usa .str.strip() para asegurar que no haya espacios en blanco en los campos
    try:
        mascara_encabezados_normativos = (df_total[1].str.strip() == '1') & \
                                         (df_total[2].str.strip() == '1') & \
                                         (df_total[3].str.strip() == 'NI') & \
                                         (df_total[4].str.strip() == '891856000')
        # Se eliminan las filas que coinciden con la máscara
        df_total = df_total[~mascara_encabezados_normativos]
    except KeyError:
        print(f"No se pudo aplicar el filtro en el año {año}. Las columnas pueden tener un formato inesperado. Se continuará con el consolidado tal como está.")


    # Renombrar las columnas a COL1, COL2, ...
    df_total.columns = [f'COL{i+1}' for i in range(df_total.shape[1])]

    # Guardar el resultado en el archivo de salida para el año actual
    df_total.to_csv(R_Salida_año, sep='|', index=False, header=True, encoding='ansi')
    
    print(f"Consolidado del año {año} guardado exitosamente en '{R_Salida_año}'")

print("Proceso de consolidación anual completado.")

Procesando el año: 2018
Error al leer el archivo 'EPS025_2018-3-16.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-18.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-19.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-20.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-23.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-25.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-26.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-29.txt': No columns to parse from file. Este archivo será ignorado.
Error al leer el archivo 'EPS025_2018-3-30.txt': No columns to parse from file. Este archivo será ignorado.
Erro